An example of working with TableHyperImg

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from itertools import combinations
from copy import deepcopy

import hyper_img as hyper

Pathes and constants

In [ ]:
IMG_PATH = '/home/igor/projects/Seed/data/hyper_img_data/'
URL = 'https://docs.google.com/spreadsheets/d/1-C3XlMbsvuBdVyGzQ6eeVoTBIg9Fb9k6zUQjbzjYCtw/gviz/tq?tqx=out:csv&sheet=annotation'
TARGET_VARIABLE = 'Mutation'

Read a google table 

In [ ]:
df = hyper.get_google_table_sheets()

In [ ]:
df.head()

Get a list of TableHyperImg

In [ ]:
img_lst: list[hyper.TableHyperImg] = hyper.get_list_hyper_img(seq_names= IMG_PATH + df[df.Mutation != ''][df['ID партии'] == '2022_09_02']['Image Name'],
                                                class_name= hyper.TableHyperImg,
                                                filter=lambda x: x != '',
                                                same_samples=['purple', 'light', 'blue'],
                                                norm_seq_tg_name=['-1', '-4'],
                                                table = df,
                                                threshold_value=28.27608638,
                                                target_varible_name=TARGET_VARIABLE)
len(img_lst)

In [ ]:
img_lst_2: list[hyper.TableHyperImg] = hyper.get_list_hyper_img(seq_names= IMG_PATH + df[df.Mutation != ''][df['ID партии'] != '2022_09_02']['Image Name'],
                                                class_name=hyper.TableHyperImg,
                                                filter=lambda x: x != '',
                                                same_samples = ['purple', 'light', 'blue'],
                                                norm_seq_tg_name=['-1', '-4'],
                                                table = df,
                                                threshold_value=28.27608638,
                                                target_varible_name=TARGET_VARIABLE)
len(img_lst_2)

In [ ]:
img_lst.extend(img_lst_2)

In [ ]:
color={'blue': 'blue',
       'purple': 'purple',
       'light': 'yellow',
       'wt': 'red',
       '-1': 'green',
       '-4': 'gray',
       '-11': 'brown',
       '1': 'black'}

Example of elements and some main operation with them

In [ ]:
img_lst[0]

In [ ]:
img_lst[0] + img_lst[1]

Charts

In [ ]:
hyper.get_medians_wavelenght_graph(img_lst, color=color)

In [ ]:
hyper.get_2_pca_graph(img_lst, color=color)

In [ ]:
compare_targets=['blue', 'purple', 'light', '-4', '-1']
img_lst_1 = [deepcopy(img) for img in img_lst 
             if img.black_calibration_img_name == '2022_09_02_calibr_session_000_026_snapshot_cube.tiff'
             and img.target_varible in compare_targets]
for img in img_lst_1:
    img.target_varible = 's1_' + img.target_varible
len(img_lst_1)

In [ ]:
compare_targets=['blue', 'purple', 'light', '-4', '-1']
img_lst_2 = [deepcopy(img) for img in img_lst 
             if img.black_calibration_img_name != '2022_09_02_calibr_session_000_026_snapshot_cube.tiff'
             and img.target_varible in compare_targets]
for img in img_lst_2:
    img.target_varible = 's2_' + img.target_varible
len(img_lst_2)

In [ ]:
img_lst_1.extend(img_lst_2)
len(img_lst_1)

In [ ]:
hyper.get_medians_wavelenght_graph(img_lst_1)

In [ ]:
hyper.get_2_pca_graph(img_lst_1, download_path='without_calibr_mutation_pca.html')

In [ ]:
hyper.get_umap_graph(img_lst_1, download_path='without_calibr_mutation_umap.html')

In [ ]:
for tg in compare_targets:
    hyper.get_mean_diff_and_confident_interval_graph(img_lst_1, 's1_' + tg, 's2_' + tg, 0.95)

In [ ]:
targets = [tg for tg in df[TARGET_VARIABLE].unique() if tg not in ['', 'light', 'purple', 'blue']]
for tg_1, tg_2 in combinations(targets, 2):
    if tg_1 == tg_2:
        continue
    hyper.get_mean_diff_and_confident_interval_graph(img_lst, tg_1, tg_2, 0.99)

In [ ]:
hyper.get_2_pca_graph(img_lst, color=color)

In [ ]:
hyper.get_umap_graph(img_lst, color=color)

Some examples of getting useful pd.DataFrame (see more in get_data_funcs.py)

In [ ]:
df = hyper.get_df_graphics_medians_wavelenght(img_lst)
df.head()

In [ ]:
plt.figure(figsize=(7,5))
plt.grid()
sns.lineplot(data=df, x='Wavelength', y='Median', hue=TARGET_VARIABLE)

In [ ]:
df = hyper.get_df_medians(img_lst)
df.head()

In [ ]:
df[TARGET_VARIABLE].hist()

In [ ]:
df, explained_variance = hyper.get_df_2_pca_and_explained_variance(img_lst)
df.head()

Create annotation

In [ ]:
hyper.create_table_annotation_df(img_lst)